In [1]:
# Stdlib packages
import copy
import datetime
import glob
import json
import os
import re
from pathlib import Path

# Common Py packages
import numpy as np
from matplotlib import pyplot as plt
from prettytable import PrettyTable

# HEP packages
import gpustat
import h5py
import hist
import mplhep as hep
import xgboost as xgb
from cycler import cycler


# ML packages
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import auc, roc_curve

# Module packages
from data_processing_BDT import process_data

gpustat.print_gpustat()

plt.style.use(hep.style.CMS)
plt.rcParams.update({'font.size': 20})
cmap_petroff10 = ["#3f90da", "#ffa90e", "#bd1f01", "#94a4a2", "#832db6", "#a96b59", "#e76300", "#b9ac70", "#717581", "#92dadd"]
plt.rcParams.update({"axes.prop_cycle": cycler("color", cmap_petroff10)})

cmslpcgpu3.fnal.gov      Fri Nov  8 13:10:36 2024  555.42.06
[0] Tesla P100-PCIE-12GB | 53°C,  91 % |   308 / 12288 MB | aherrera(306M)


In [2]:
lpc_fileprefix = "/eos/uscms/store/group/lpcdihiggsboost/tsievert/HiggsDNA_parquet/v1"

FILEPATHS_DICT = {
    'ggF HH': [
        lpc_fileprefix+f"/Run3_2022preEE_merged_v3/GluGluToHH/nominal/*", 
        lpc_fileprefix+f"/Run3_2022postEE_merged_v3/GluGluToHH/nominal/*"
    ],
    # 'VBF HH': [
    #     lpc_fileprefix+f"/Run3_2022preEE_merged_v3/VBFHHto2B2G_CV_1_C2V_1_C3_1/nominal/*", 
    #     lpc_fileprefix+f"/Run3_2022postEE_merged_v3/VBFHHto2B2G_CV_1_C2V_1_C3_1/nominal/*"
    # ],
    'ttH': [
        lpc_fileprefix+f"/Run3_2022preEE_merged_v3/ttHToGG/nominal/*", 
        lpc_fileprefix+f"/Run3_2022postEE_merged_v3/ttHToGG/nominal/*"
    ],
    'non-res + single-H': [
        # non-Resonant #
        # GG + 3Jets
        lpc_fileprefix+f"/Run3_2022preEE_merged_v3/GGJets/nominal/*", 
        lpc_fileprefix+f"/Run3_2022postEE_merged_v3/GGJets/nominal/*",
        # GJet pT 20-40
        lpc_fileprefix+f"/Run3_2022preEE_merged_v3/GJetPt20To40/nominal/*", 
        lpc_fileprefix+f"/Run3_2022postEE_merged_v3/GJetPt20To40/nominal/*",
        # GJet pT 40-inf
        lpc_fileprefix+f"/Run3_2022preEE_merged_v3/GJetPt40/nominal/*", 
        lpc_fileprefix+f"/Run3_2022postEE_merged_v3/GJetPt40/nominal/*",
        # single-H #
        # ggF H
        lpc_fileprefix+f"/Run3_2022preEE_merged_v3/GluGluHToGG/nominal/*", 
        lpc_fileprefix+f"/Run3_2022postEE_merged_v3/GluGluHToGG/nominal/*",
        # VBF H
        lpc_fileprefix+f"/Run3_2022preEE_merged_v3/VBFHToGG/nominal/*", 
        lpc_fileprefix+f"/Run3_2022postEE_merged_v3/VBFHToGG/nominal/*",
        # VH
        lpc_fileprefix+f"/Run3_2022preEE_merged_v3/VHToGG/nominal/*", 
        lpc_fileprefix+f"/Run3_2022postEE_merged_v3/VHToGG/nominal/*",
    ],
    # 'VH': [
    #     lpc_fileprefix+f"/Run3_2022preEE_merged_v3/VHToGG/nominal/*", 
    #     lpc_fileprefix+f"/Run3_2022postEE_merged_v3/VHToGG/nominal/*"
    # ],
}

CURRENT_DIRPATH = str(Path().absolute())
VERSION = 'v1'
MOD_VALS = (5, 5)
VARS = 'nonres_and_ttH_vars'
# OUTPUT_DIRPATH = os.path.join(CURRENT_DIRPATH, f"MultiClassBDT_model_outputs/{VERSION}/{VARS}", CURRENT_TIME)
OUTPUT_DIRPATH = os.path.join(CURRENT_DIRPATH, f"MultiClassBDT_model_outputs/{VERSION}/{VARS}")

if not os.path.exists(OUTPUT_DIRPATH):
    os.makedirs(OUTPUT_DIRPATH)

SEED = 21
OPTIMIZE_SPACE = False
NUM_EPOCHS = 150

In [3]:
def training_weights(event_weights, labels, order=None):
    if order is None:
        order = [i for i in range(np.shape(labels)[0])]
    sum_dict, max_sum, max_i = {}, 0, 0
    for i, sample_name in enumerate(order):
        sum_dict[i] = np.sum(event_weights[labels[:, i] == 1])
        if np.sum(event_weights[labels[:, i] == 1]) > max_sum:
            max_sum, max_i = np.sum(event_weights[labels[:, i] == 1]), i

    label_i = np.sum(
        np.tile([i for i in range(np.shape(labels)[1])], (np.shape(labels)[0], 1)) * labels,
        axis=1
    )

    weight_factors = []
    for i in range(len(label_i)):
        weight_factors.append(
            max_sum / sum_dict[label_i[i]] if label_i[i] != max_i else 1
        )
    weights = event_weights * np.array(weight_factors)

    mean_weight = np.mean(weights)
    abs_weights = np.abs(weights)
    scaled_weights = abs_weights / mean_weight

    return scaled_weights

def xgb_labels(labels):
    label_i = np.sum(
        np.tile([i for i in range(np.shape(labels)[1])], (np.shape(labels)[0], 1)) * labels,
        axis=1
    )

    return label_i

In [4]:
order = ['ggF HH', 'ttH', 'non-res + single-H']

(
    data_df_dict, data_test_df_dict, 
    data_hlf_dict, label_dict, 
    data_hlf_test_dict, label_test_dict, 
    high_level_fields_dict, input_hlf_vars_dict, hlf_vars_columns_dict,
    data_aux_dict, data_test_aux_dict
) = process_data(
    FILEPATHS_DICT, OUTPUT_DIRPATH, order=order, seed=SEED, mod_vals=MOD_VALS, k_fold_test=True
)

xgb_label_dict = {
    f"fold_{fold_idx}": copy.deepcopy(xgb_labels(label_dict[f"fold_{fold_idx}"])) for fold_idx in range(len(data_test_aux_dict))
}
xgb_label_test_dict = {
    f"fold_{fold_idx}": copy.deepcopy(xgb_labels(label_test_dict[f"fold_{fold_idx}"])) for fold_idx in range(len(data_test_aux_dict))
}

weight_train_dict = {
    f"fold_{fold_idx}": copy.deepcopy(training_weights(data_aux_dict[f'fold_{fold_idx}'].loc[:, "eventWeight"].to_numpy(), label_dict[f'fold_{fold_idx}'], order=order)) for fold_idx in range(len(data_test_aux_dict))
}
weight_test_dict = {
    f'fold_{fold_idx}': copy.deepcopy(data_test_aux_dict[f'fold_{fold_idx}'].loc[:, "eventWeight"].to_numpy()) for fold_idx in range(len(data_test_aux_dict))
}

Data HLF: (1563192, 48)
num ggF HH = 136530
num ttH = 277205
num non-res + single-H = 1149457
Data HLF test: (391785, 48)
num ggF HH = 34224
num ttH = 69297
num non-res + single-H = 288264
Data HLF: (1564171, 48)
num ggF HH = 136466
num ttH = 277452
num non-res + single-H = 1150253
Data HLF test: (390806, 48)
num ggF HH = 34288
num ttH = 69050
num non-res + single-H = 287468
Data HLF: (1563685, 48)
num ggF HH = 136638
num ttH = 276627
num non-res + single-H = 1150420
Data HLF test: (391292, 48)
num ggF HH = 34116
num ttH = 69875
num non-res + single-H = 287301
Data HLF: (1564419, 48)
num ggF HH = 136671
num ttH = 277054
num non-res + single-H = 1150694
Data HLF test: (390558, 48)
num ggF HH = 34083
num ttH = 69448
num non-res + single-H = 287027
Data HLF: (1564441, 48)
num ggF HH = 136711
num ttH = 277670
num non-res + single-H = 1150060
Data HLF test: (390536, 48)
num ggF HH = 34043
num ttH = 68832
num non-res + single-H = 287661


In [7]:
bdt_train_dict, bdt_val_dict, bdt_test_dict = {}, {}, {}
for fold_idx in range(len(data_df_dict)):
    if re.search('no_std', VARS) is not None:
        print('no standardization')
        train_val_data_dict = {key: value.to_numpy() for key, value in data_df_dict.items()}
        test_data_dict = {key: value.to_numpy() for key, value in data_test_df_dict.items()}
    else:
        train_val_data_dict = data_hlf_dict
        test_data_dict = data_hlf_test_dict
    (
        X_train, X_val, y_train, y_val, weight_train, weight_val
    ) = train_test_split(
        train_val_data_dict[f"fold_{fold_idx}"], xgb_label_dict[f"fold_{fold_idx}"], weight_train_dict[f"fold_{fold_idx}"],
        test_size=0.2, random_state=21
    )

    bdt_train_dict[f"fold_{fold_idx}"] = xgb.DMatrix(
        data=X_train, label=y_train, 
        weight=weight_train,
        missing=-999.0, feature_names=list(high_level_fields_dict[f"fold_{fold_idx}"])
    )
    bdt_val_dict[f"fold_{fold_idx}"] = xgb.DMatrix(
        data=X_val, label=y_val, 
        weight=weight_val,
        missing=-999.0, feature_names=list(high_level_fields_dict[f"fold_{fold_idx}"])
    )
    
    bdt_test_dict[f"fold_{fold_idx}"] = xgb.DMatrix(
        data=test_data_dict[f"fold_{fold_idx}"], label=xgb_label_test_dict[f"fold_{fold_idx}"], 
        weight=np.abs(weight_test_dict[f"fold_{fold_idx}"]),
        missing=-999.0, feature_names=list(high_level_fields_dict[f"fold_{fold_idx}"])
    )

    print(f"fold {fold_idx}")
    print(f"Num train: {len(y_train)} -> {sum(y_train == 0)} sig & {sum(y_train == 1)} ttH bkg & {sum(y_train == 2)} non-res + single-H bkg")
    print(f"Num val: {len(y_val)} -> {sum(y_val == 0)} sig & {sum(y_val == 1)} ttH bkg & {sum(y_val == 2)} non-res + single-H bkg")
    print(f"Num test: {len(label_test_dict[f'fold_{fold_idx}'])} -> {sum(label_test_dict[f'fold_{fold_idx}'] == np.array([1, 0, 0]))[0]} sig & {sum(label_test_dict[f'fold_{fold_idx}'] == np.array([0, 1, 0]))[1]} ttH bkg & {sum(label_test_dict[f'fold_{fold_idx}'] == np.array([0, 0, 1]))[2]} non-res + single-H bkg")
    print('='*60)

    

fold 0
Num train: 1250553 -> 109096 sig & 222178 ttH bkg & 919279 non-res + single-H bkg
Num val: 312639 -> 27434 sig & 55027 ttH bkg & 230178 non-res + single-H bkg
Num test: 391785 -> 34224 sig & 69297 ttH bkg & 288264 non-res + single-H bkg
fold 1
Num train: 1251336 -> 109298 sig & 221622 ttH bkg & 920416 non-res + single-H bkg
Num val: 312835 -> 27168 sig & 55830 ttH bkg & 229837 non-res + single-H bkg
Num test: 390806 -> 34288 sig & 69050 ttH bkg & 287468 non-res + single-H bkg
fold 2
Num train: 1250948 -> 109478 sig & 221207 ttH bkg & 920263 non-res + single-H bkg
Num val: 312737 -> 27160 sig & 55420 ttH bkg & 230157 non-res + single-H bkg
Num test: 391292 -> 34116 sig & 69875 ttH bkg & 287301 non-res + single-H bkg
fold 3
Num train: 1251535 -> 109266 sig & 221688 ttH bkg & 920581 non-res + single-H bkg
Num val: 312884 -> 27405 sig & 55366 ttH bkg & 230113 non-res + single-H bkg
Num test: 390558 -> 34083 sig & 69448 ttH bkg & 287027 non-res + single-H bkg
fold 4
Num train: 125155

In [8]:
for i in range(5):
    print(f'fold {i}: max = {np.max(weight_train_dict[f"fold_{i}"]):.4f}, min = {np.min(weight_train_dict[f"fold_{i}"]):.4f}, mean ± std. dev. = {np.mean(weight_train_dict[f"fold_{i}"]):.4f} ± {np.std(weight_train_dict[f"fold_{i}"]):.4f}')

fold 0: max = 27.1178, min = 0.0016, mean ± std. dev. = 1.6918 ± 2.5556
fold 1: max = 27.1368, min = 0.0016, mean ± std. dev. = 1.6972 ± 2.5651
fold 2: max = 27.1991, min = 0.0016, mean ± std. dev. = 1.6869 ± 2.5528
fold 3: max = 27.1529, min = 0.0016, mean ± std. dev. = 1.6906 ± 2.5552
fold 4: max = 27.0900, min = 0.0016, mean ± std. dev. = 1.6887 ± 2.5516


In [9]:
# https://stackoverflow.com/questions/57986259/multiclass-classification-with-xgboost-classifier
# https://forecastegy.com/posts/xgboost-multiclass-classification-python/
# https://indico.cern.ch/event/915265/contributions/3848138/attachments/2048174/3432202/kunlinRan_bbyy_20200531.pdf


param = {}

# Booster parameters
param['eta']              = 0.01 # learning rate
param['max_depth']        = 5  # maximum depth of a tree
param['subsample']        = 0.6 # fraction of events to train tree on
param['colsample_bytree'] = 0.4 # fraction of features to train tree on
param['num_class']        = np.shape(label_dict['fold_0'])[1] # num classes for ulti-class training

# Learning task parameters
param['objective']   = 'multi:softprob'   # objective function
param['eval_metric'] = 'merror'           # evaluation metric for cross validation
param = list(param.items()) + [('eval_metric', 'mlogloss')]

num_trees = 1000  # number of trees to make

In [10]:
CURRENT_TIME = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
OUTPUT_DIRPATH = os.path.join(OUTPUT_DIRPATH, CURRENT_TIME)
if not os.path.exists(OUTPUT_DIRPATH):
    os.makedirs(OUTPUT_DIRPATH)

for fold_idx in range(len(bdt_train_dict)):
    print(f"fold {fold_idx}")
    # Train bdt
    evallist = [(bdt_train_dict[f"fold_{fold_idx}"], 'train'), (bdt_test_dict[f"fold_{fold_idx}"], 'test'), (bdt_val_dict[f"fold_{fold_idx}"], 'val')]
    booster = xgb.train(
        param, bdt_train_dict[f"fold_{fold_idx}"], num_boost_round=num_trees, 
        evals=evallist, early_stopping_rounds=7, verbose_eval=True
    )
    booster.save_model(os.path.join(OUTPUT_DIRPATH, f'{CURRENT_TIME}_BDT_fold{fold_idx}.model'))
    
    # Print perf on test dataset
    print(booster.eval(bdt_test_dict[f"fold_{fold_idx}"]))
    print('='*100)

fold 0
[0]	train-merror:0.15501	train-mlogloss:1.08970	test-merror:0.17075	test-mlogloss:1.08989	val-merror:0.15614	val-mlogloss:1.08998
[1]	train-merror:0.16322	train-mlogloss:1.08243	test-merror:0.21088	test-mlogloss:1.08380	val-merror:0.16439	val-mlogloss:1.08268
[2]	train-merror:0.16656	train-mlogloss:1.07595	test-merror:0.21457	test-mlogloss:1.07936	val-merror:0.16715	val-mlogloss:1.07620
[3]	train-merror:0.16152	train-mlogloss:1.06859	test-merror:0.21415	test-mlogloss:1.07436	val-merror:0.16270	val-mlogloss:1.06884
[4]	train-merror:0.15537	train-mlogloss:1.06068	test-merror:0.20881	test-mlogloss:1.06805	val-merror:0.15660	val-mlogloss:1.06095
[5]	train-merror:0.15721	train-mlogloss:1.05383	test-merror:0.21658	test-mlogloss:1.06244	val-merror:0.15803	val-mlogloss:1.05410
[6]	train-merror:0.15226	train-mlogloss:1.04656	test-merror:0.19818	test-mlogloss:1.05557	val-merror:0.15337	val-mlogloss:1.04683
[7]	train-merror:0.14248	train-mlogloss:1.03782	test-merror:0.17727	test-mlogloss:1

In [15]:
# OUTPUT_DIRPATH = os.path.join(OUTPUT_DIRPATH, CURRENT_TIME)
# if not os.path.exists(OUTPUT_DIRPATH):
#     os.makedirs(OUTPUT_DIRPATH)

BDT_perf = {
    'preds': [], 'fprs': [], 'tprs': [], 'thresholds': [], 'areas': [],
    'train_preds': [], 'train_fprs': [], 'train_tprs': [], 'train_thresholds': [], 'train_areas': [],
    'val_preds': [], 'val_fprs': [], 'val_tprs': [], 'val_thresholds': [], 'val_areas': [],
}
base_tpr = np.linspace(0, 1, 5000)  # copied from IN evaluate.py file
for fold_idx in range(len(bdt_train_dict)):
    booster = xgb.Booster(param)
    booster.load_model(os.path.join(OUTPUT_DIRPATH, f'{CURRENT_TIME}_BDT_fold{fold_idx}.model'))
    
    for pred_type, dataset in [
        ('train_', bdt_train_dict[f"fold_{fold_idx}"]), 
        ('val_', bdt_val_dict[f"fold_{fold_idx}"]),
        ('', bdt_test_dict[f"fold_{fold_idx}"])
    ]:
        BDT_perf[pred_type + 'preds'].append(booster.predict(dataset).tolist())
        print(np.shape(dataset.get_label()))
        print(np.all(dataset.get_label() == dataset.get_label()[0]))
        print(np.shape(BDT_perf[pred_type + 'preds'][-1]))
        print(np.all(BDT_perf[pred_type + 'preds'][-1][:][0] == BDT_perf[pred_type + 'preds'][-1][0][0]))
        print(np.all(BDT_perf[pred_type + 'preds'][-1][:][1] == BDT_perf[pred_type + 'preds'][-1][0][1]))
        print(np.all(BDT_perf[pred_type + 'preds'][-1][:][2] == BDT_perf[pred_type + 'preds'][-1][0][2]))
        print(np.array(BDT_perf[pred_type + 'preds'][-1])[np.array(BDT_perf[pred_type + 'preds'][-1])[:,0] != BDT_perf[pred_type + 'preds'][-1][0][0]])
        print(np.array(BDT_perf[pred_type + 'preds'][-1])[np.array(BDT_perf[pred_type + 'preds'][-1])[:,1] != BDT_perf[pred_type + 'preds'][-1][0][1]])
        print(np.array(BDT_perf[pred_type + 'preds'][-1])[np.array(BDT_perf[pred_type + 'preds'][-1])[:,2] != BDT_perf[pred_type + 'preds'][-1][0][2]])
        fpr_bdt, tpr_bdt, threshold_bdt = roc_curve(dataset.get_label(), BDT_perf[pred_type + 'preds'][-1])
        fpr_bdt = np.interp(base_tpr, tpr_bdt, fpr_bdt)
        threshold_bdt = np.interp(base_tpr, tpr_bdt, threshold_bdt)
        BDT_perf[pred_type + 'fprs'].append(fpr_bdt.tolist())
        BDT_perf[pred_type + 'tprs'].append(tpr_bdt.tolist())
        BDT_perf[pred_type + 'thresholds'].append(threshold_bdt.tolist())

        BDT_perf[pred_type + 'areas'].append(float(auc(fpr_bdt, base_tpr)))

    with h5py.File(os.path.join(OUTPUT_DIRPATH, CURRENT_TIME+f"_BDT_ROC_fold{fold_idx}.h5"), "w") as out:
        out['FPR'] = fpr_bdt
        out['TPR'] = tpr_bdt
        out['Thresholds'] = threshold_bdt

with open(os.path.join(OUTPUT_DIRPATH, CURRENT_TIME+"_BDT_perf.json"), 'w') as f:
    json.dump(BDT_perf, f)

(1250553,)
True
(1250553, 3)
False
False
False
[]
[]
[]


ValueError: y should be a 1d array, got an array of shape (1250553, 3) instead.

In [ ]:
def plot_filepath(plot_name, plot_dirpath, plot_prefix, plot_postfix, format='png'):
    plot_prefix = plot_prefix + ('_' if plot_prefix != '' else '')
    plot_postfix = plot_postfix + ('_' if plot_postfix != '' else '')
    plot_name = plot_prefix + plot_name + plot_postfix + f'.{format}'

    plot_filepath = os.path.join(plot_dirpath, plot_name)
    return plot_filepath

def plot_rocs(
    fprs, tprs, labels, plot_name, plot_dirpath,
    plot_prefix='', plot_postfix='', close=True, log=None
):
    plt.figure(figsize=(9,7))
    
    for fpr, tpr, label in zip(fprs, tprs, labels):
        linestyle = 'solid' if re.search('IN', label) is not None else ('dashed' if re.search('BDT', label) is not None else 'dotted')
        plt.plot(fpr, tpr, label=label, linestyle=linestyle)

    plt.legend(bbox_to_anchor=(1, 1))
    plt.xlabel('Background contamination')
    plt.ylabel('Signal efficiency')
    if log is not None and re.search('x', log) is not None:
        plt.xscale('log')
    elif log is not None and re.search('y', log) is not None:
        plt.yscale('log')
    
    plt.savefig(
        plot_filepath(plot_name, plot_dirpath, plot_prefix, plot_postfix), 
        bbox_inches='tight'
    )
    plt.savefig(
        plot_filepath(plot_name, plot_dirpath, plot_prefix, plot_postfix, format='pdf'), 
        bbox_inches='tight'
    )
    if close:
        plt.close()

def plot_output_scores(
    sigs, bkgs, labels, plot_name, plot_dirpath,
    plot_prefix='', plot_postfix='', bins=50, weights={'sig': None, 'bkg': None}
):
    plt.figure(figsize=(9,7))

    for sig, bkg, label in zip(sigs, bkgs, labels):
        hist_axis = hist.axis.Regular(bins, 0., 1., name='var', growth=False, underflow=False, overflow=False)
        sig_hist = hist.Hist(hist_axis, storage='weight').fill(var=sig, weight=weights['sig'] if weights['sig'] is not None else np.ones_like(sig))
        bkg_hist = hist.Hist(hist_axis, storage='weight').fill(var=bkg, weight=weights['bkg'] if weights['bkg'] is not None else np.ones_like(bkg))
        hep.histplot(
            [sig_hist, bkg_hist],
            yerr=(True if weights['sig'] is not None else False),
            alpha=0.7, density=(False if weights['sig'] is not None else True), histtype='step',
            label=[f'{label} - HH signal', f'{label} - ttH background']
        )

    plt.legend(bbox_to_anchor=(1, 1))
    plt.xlabel('Output score')
    
    plt.savefig(
        plot_filepath(plot_name, plot_dirpath, plot_prefix, plot_postfix), 
        bbox_inches='tight'
    )
    plt.savefig(
        plot_filepath(plot_name, plot_dirpath, plot_prefix, plot_postfix, format='pdf'), 
        bbox_inches='tight'
    )
    plt.close()

def plot_s_over_root_b(
    sigs, bkgs, labels, plot_name, plot_dirpath,
    plot_prefix='', plot_postfix='', bins=50, weights={'sig': None, 'bkg': None},
    lines=None, lines_labels=None, line_colors=None
):
    plt.figure(figsize=(9,7))

    for idx, (sig, bkg, label) in enumerate(zip(sigs, bkgs, labels)):
        linestyle = 'solid' if re.search('IN', label) is not None else ('dashed' if re.search('BDT', label) is not None else 'dotted')

        hist_axis = hist.axis.Regular(bins, 0., 1., name='var', growth=False, underflow=False, overflow=False)
        sig_hist = hist.Hist(hist_axis, storage='weight').fill(var=sig, weight=weights['sig'] if weights['sig'] is not None else np.ones_like(sig))
        bkg_hist = hist.Hist(hist_axis, storage='weight').fill(var=bkg, weight=weights['bkg'] if weights['bkg'] is not None else np.ones_like(bkg))
        s_over_root_b_points = sig_hist.values().flatten() / np.sqrt(bkg_hist.values().flatten())
        plt.plot(
            np.arange(0., 1., 1/bins), s_over_root_b_points, 
            label=f'{label} - s/√b', alpha=0.8, linestyle=linestyle
        )

        if lines is not None:
            for i in range(len(lines[idx])):
                plt.vlines(
                    lines[idx][i], 0, np.max(s_over_root_b_points), 
                    label='s/√b'+(' - '+lines_labels[idx][i] if lines_labels is not None else ''), 
                    alpha=0.5, linestyle=linestyle, colors=line_colors[idx][i]
                )
    
    plt.legend(bbox_to_anchor=(1, 1))
    plt.xlabel('Output score')
    plt.ylabel('s/√b')
    
    plt.savefig(
        plot_filepath(plot_name, plot_dirpath, plot_prefix, plot_postfix), 
        bbox_inches='tight'
    )
    plt.savefig(
        plot_filepath(plot_name, plot_dirpath, plot_prefix, plot_postfix, format='pdf'), 
        bbox_inches='tight'
    )
    plt.close()

In [ ]:
def optimize_cut_boundaries(sigs, bkgs, weights, bins=50):
    hist_list_fold = []
    cut_boundaries_fold = []
    cut_s_over_root_bs_fold = []
    sig_weights_fold = []
    bkg_weights_fold = []
    for sig, bkg in zip(sigs, bkgs):
        hist_axis = hist.axis.Regular(bins, 0., 1., name='var', growth=False, underflow=False, overflow=False)
        sig_hist = hist.Hist(hist_axis, storage='weight').fill(var=sig, weight=weights['sig'])
        bkg_hist = hist.Hist(hist_axis, storage='weight').fill(var=bkg, weight=weights['bkg'])
        hist_list_fold.append({'sig': copy.deepcopy(sig_hist), 'bkg': copy.deepcopy(bkg_hist)})

        fold_idx_cuts_bins_inclusive = []
        fold_idx_sig_weights = []
        fold_idx_bkg_weights = []
        fold_idx_prev_s_over_root_b = []
        prev_s_over_root_b = 0
        for i in range(bins):
            s = np.sum(sig_hist.values().flatten()[
                (bins-1) - i : bins if len(fold_idx_cuts_bins_inclusive) == 0 else fold_idx_cuts_bins_inclusive[-1]
            ])
            sqrt_b = np.sqrt(np.sum(bkg_hist.values().flatten()[
                (bins-1) - i : bins if len(fold_idx_cuts_bins_inclusive) == 0 else fold_idx_cuts_bins_inclusive[-1]
            ]))
            if prev_s_over_root_b < (s / sqrt_b):
                prev_s_over_root_b = s / sqrt_b
                continue
            else:
                fold_idx_sig_weights.append(
                    {
                        'value': np.sum(sig_hist.values().flatten()[
                            (bins) - i : bins if len(fold_idx_cuts_bins_inclusive) == 0 else fold_idx_cuts_bins_inclusive[-1]
                        ]),
                        'w2': np.sqrt(np.sum(sig_hist.variances().flatten()[
                            (bins) - i : bins if len(fold_idx_cuts_bins_inclusive) == 0 else fold_idx_cuts_bins_inclusive[-1]
                        ])),
                    }
                )
                fold_idx_bkg_weights.append(
                    {
                        'value': np.sum(bkg_hist.values().flatten()[
                            (bins) - i : bins if len(fold_idx_cuts_bins_inclusive) == 0 else fold_idx_cuts_bins_inclusive[-1]
                        ]),
                        'w2': np.sqrt(np.sum(bkg_hist.variances().flatten()[
                            (bins) - i : bins if len(fold_idx_cuts_bins_inclusive) == 0 else fold_idx_cuts_bins_inclusive[-1]
                        ])),
                    }
                )
                fold_idx_cuts_bins_inclusive.append(bins - i)
                fold_idx_prev_s_over_root_b.append(prev_s_over_root_b)
                prev_s_over_root_b = 0
        fold_idx_sig_weights.append(
            {
                'value': np.sum(sig_hist.values().flatten()[
                    0 : bins if len(fold_idx_cuts_bins_inclusive) == 0 else fold_idx_cuts_bins_inclusive[-1]
                ]),
                'w2': np.sqrt(np.sum(sig_hist.variances().flatten()[
                    0 : bins if len(fold_idx_cuts_bins_inclusive) == 0 else fold_idx_cuts_bins_inclusive[-1]
                ])),
            }
        )
        fold_idx_bkg_weights.append(
            {
                'value': np.sum(bkg_hist.values().flatten()[
                    0 : bins if len(fold_idx_cuts_bins_inclusive) == 0 else fold_idx_cuts_bins_inclusive[-1]
                ]),
                'w2': np.sqrt(np.sum(bkg_hist.variances().flatten()[
                    0 : bins if len(fold_idx_cuts_bins_inclusive) == 0 else fold_idx_cuts_bins_inclusive[-1]
                ])),
            }
        )
        fold_idx_cuts_bins_inclusive.append(0)
        fold_idx_prev_s_over_root_b.append(prev_s_over_root_b)
        fold_idx_score_cuts = [bin_i / bins for bin_i in fold_idx_cuts_bins_inclusive]
        cut_boundaries_fold.append(fold_idx_score_cuts)
        cut_s_over_root_bs_fold.append(fold_idx_prev_s_over_root_b)
        sig_weights_fold.append(fold_idx_sig_weights)
        bkg_weights_fold.append(fold_idx_bkg_weights)
    return cut_boundaries_fold, cut_s_over_root_bs_fold, sig_weights_fold, bkg_weights_fold

In [ ]:
# OUTPUT_DIRPATH = os.path.join(OUTPUT_DIRPATH, CURRENT_TIME)
plot_dirpath = os.path.join(OUTPUT_DIRPATH, "plots")
if not os.path.exists(plot_dirpath):
    os.makedirs(plot_dirpath)

with open('/uscms/home/tsievert/nobackup/XHYbbgg/HHtobbyy/model_outputs/v8/extra_vars+/2024-10-09_20-47-24_IN_perf.json', 'r') as f:
    IN_perf = json.load(f)

# plot ROCs
avg_fprs, avg_aucs = [], []
for fold_idx in range(len(bdt_train_dict)):
    fprs = [BDT_perf['fprs'][fold_idx], IN_perf['fprs'][fold_idx]]
    avg_fprs.append(copy.deepcopy(fprs))
    tprs = [base_tpr] * 2
    labels = [f"BDT, AUC = {BDT_perf['areas'][fold_idx]:.4f}", f"IN, AUC = {auc(IN_perf['fprs'][fold_idx], base_tpr):.4f}"]
    avg_aucs.append(copy.deepcopy([BDT_perf['areas'][fold_idx], auc(IN_perf['fprs'][fold_idx], base_tpr)]))

    plot_rocs(fprs, tprs, labels, f"BDT_IN_roc_testData_fold{fold_idx}", plot_dirpath)
plot_rocs(
    np.mean(avg_fprs, axis=0), [base_tpr] * 2, 
    [f"BDT, AUC = {np.mean(avg_aucs, axis=0)[0]:.4f}", f"IN, AUC = {np.mean(avg_aucs, axis=0)[1]:.4f}"], 
    f"BDT_IN_roc_testData_Avg", plot_dirpath
)

# plot Output scores
for fold_idx in range(len(bdt_train_dict)):
    #     'sig': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 1) & mask_arr[fold_idx]],
    #     'bkg': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 0) & mask_arr[fold_idx]],
    weights_plot = {
        'sig': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 1)],
        'bkg': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 0)],
    }
    
    sigs = [
        np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 1],
        np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 1, 1]
    ]
    bkgs = [
        np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 0],
        np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 0, 1]
    ]
    labels = ["BDT", "IN"]

    plot_output_scores(sigs, bkgs, labels, f"BDT_IN_outputScoreWeighted_testData_fold{fold_idx}", plot_dirpath, weights=weights_plot)
    plot_output_scores(sigs, bkgs, labels, f"BDT_IN_outputScoreDensity_testData_fold{fold_idx}", plot_dirpath)

# plot s/√b curves
for fold_idx in range(len(bdt_train_dict)):
    weights_plot = {
        'sig': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 1)],
        'bkg': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 0)],
    }
    
    sigs = [
        np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 1],
        np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 1, 1]
    ]
    bkgs = [
        np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 0],
        np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 0, 1]
    ]
    labels = ["BDT", "IN"]

    plot_s_over_root_b(sigs, bkgs, labels, f"BDT_IN_sOverRootb_testData_fold{fold_idx}", plot_dirpath, weights=weights_plot)

for fold_idx in range(len(bdt_train_dict)):
    weights_plot = {
        'sig': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 1)],
        'bkg': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 0)],
    }
    
    sigs = [
        np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 1],
        np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 1, 1]
    ]
    bkgs = [
        np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 0],
        np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 0, 1]
    ]
    labels = ["BDT", "IN"]

    (
        cut_boundaries_fold, cut_s_over_root_bs_fold, sig_weights_fold, bkg_weights_fold
    ) = optimize_cut_boundaries(sigs, bkgs, weights_plot)

    BDT_cut_labels = [
        f"s/√b={cut_s_over_root_bs_fold[0][cut_idx]:.4f}, s={sig_weights_fold[0][cut_idx]['value']:.4f}±{sig_weights_fold[0][cut_idx]['w2']:.4f}, b={bkg_weights_fold[0][cut_idx]['value']:.4f}±{bkg_weights_fold[0][cut_idx]['w2']:.4f}" for cut_idx in range(len(cut_s_over_root_bs_fold[0]))
    ]
    IN_cut_labels = [
        f"s/√b={cut_s_over_root_bs_fold[1][cut_idx]:.4f}, s={sig_weights_fold[1][cut_idx]['value']:.4f}±{sig_weights_fold[1][cut_idx]['w2']:.4f}, b={bkg_weights_fold[1][cut_idx]['value']:.4f}±{bkg_weights_fold[1][cut_idx]['w2']:.4f}" for cut_idx in range(len(cut_s_over_root_bs_fold[1]))
    ]
    line_labels = [BDT_cut_labels, IN_cut_labels]
    lines = [cut_boundaries_fold[0], cut_boundaries_fold[1]]
    line_colors=[cmap_petroff10, cmap_petroff10]

    plot_s_over_root_b(
        sigs, bkgs, labels, f"BDT_IN_sOverRootb_withCuts_testData_fold{fold_idx}", plot_dirpath, weights=weights_plot,
        lines=lines, lines_labels=line_labels, line_colors=line_colors
    )

In [ ]:
# OUTPUT_DIRPATH = os.path.join(OUTPUT_DIRPATH, CURRENT_TIME)
plot_dirpath = os.path.join(OUTPUT_DIRPATH, "plots")
if not os.path.exists(plot_dirpath):
    os.makedirs(plot_dirpath)

with open('/uscms/home/tsievert/nobackup/XHYbbgg/HHtobbyy/model_outputs/v8/extra_vars+/2024-10-09_20-47-24_IN_perf.json', 'r') as f:
    IN_perf = json.load(f)

# plot ROCs
avg_fprs, avg_aucs = [], []
for fold_idx in range(len(bdt_train_dict)):
    fprs = [
        BDT_perf['train_fprs'][fold_idx], 
        BDT_perf['val_fprs'][fold_idx],
        BDT_perf['fprs'][fold_idx]
    ]
    avg_fprs.append(copy.deepcopy(fprs))
    tprs = [base_tpr] * 3
    labels = [
        f"BDT train data fold {fold_idx}, AUC = {BDT_perf['train_areas'][fold_idx]:.4f}", 
        f"BDT val data fold {fold_idx}, AUC = {BDT_perf['val_areas'][fold_idx]:.4f}",
        f"BDT test data fold {fold_idx}, AUC = {BDT_perf['areas'][fold_idx]:.4f}"
    ]
    avg_aucs.append(copy.deepcopy([
        BDT_perf['train_areas'][fold_idx], 
        BDT_perf['val_areas'][fold_idx],
        BDT_perf['areas'][fold_idx]
    ]))

    plot_rocs(fprs, tprs, labels, f"BDT_roc_allData_fold{fold_idx}", plot_dirpath, close=False, log='x')
plot_rocs(
    np.mean(avg_fprs, axis=0), [base_tpr] * 3, 
    [
        f"BDT train avg., AUC = {np.mean(avg_aucs, axis=0)[0]:.4f}", 
        f"BDT val avg., AUC = {np.mean(avg_aucs, axis=0)[1]:.4f}",
        f"BDT test avg., AUC = {np.mean(avg_aucs, axis=0)[2]:.4f}"
    ], 
    f"BDT_roc_allData_Avg", plot_dirpath, close=False, log='x'
)

# plot Output scores
# for fold_idx in range(len(bdt_train_dict)):
#     #     'sig': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 1) & mask_arr[fold_idx]],
#     #     'bkg': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 0) & mask_arr[fold_idx]],
#     weights_plot = {
#         'sig': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 1)],
#         'bkg': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 0)],
#     }
    
#     sigs = [
#         np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 1],
#         np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 1, 1]
#     ]
#     bkgs = [
#         np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 0],
#         np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 0, 1]
#     ]
#     labels = ["BDT", "IN"]

#     plot_output_scores(sigs, bkgs, labels, f"BDT_IN_outputScoreWeighted_testData_fold{fold_idx}", plot_dirpath, weights=weights_plot)
#     plot_output_scores(sigs, bkgs, labels, f"BDT_IN_outputScoreDensity_testData_fold{fold_idx}", plot_dirpath)

# # plot s/√b curves
# for fold_idx in range(len(bdt_train_dict)):
#     weights_plot = {
#         'sig': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 1)],
#         'bkg': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 0)],
#     }
    
#     sigs = [
#         np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 1],
#         np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 1, 1]
#     ]
#     bkgs = [
#         np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 0],
#         np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 0, 1]
#     ]
#     labels = ["BDT", "IN"]

#     plot_s_over_root_b(sigs, bkgs, labels, f"BDT_IN_sOverRootb_testData_fold{fold_idx}", plot_dirpath, weights=weights_plot)

# # for fold_idx in range(len(bdt_train_dict)):
# #     weights_plot = {
# #         'sig': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 1)],
# #         'bkg': weight_test_dict[f"fold_{fold_idx}"][(label_test_dict[f"fold_{fold_idx}"] == 0)],
# #     }
    
# #     sigs = [
# #         np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 1],
# #         np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 1, 1]
# #     ]
# #     bkgs = [
# #         np.array(BDT_perf['preds'][fold_idx])[label_test_dict[f"fold_{fold_idx}"] == 0],
# #         np.exp(np.array(IN_perf['all_preds'][fold_idx]))[np.array(IN_perf['all_labels'][fold_idx]) == 0, 1]
# #     ]
# #     labels = ["BDT", "IN"]

# #     (
# #         cut_boundaries_fold, cut_s_over_root_bs_fold, sig_weights_fold, bkg_weights_fold
# #     ) = optimize_cut_boundaries(sigs, bkgs, weights_plot)

# #     BDT_cut_labels = [
# #         f"s/√b={cut_s_over_root_bs_fold[0][cut_idx]:.2f}, s={sig_weights_fold[0][cut_idx]['value']:.2f}±{sig_weights_fold[0][cut_idx]['w2']:.2f}, b={bkg_weights_fold[0][cut_idx]['value']:.2f}±{bkg_weights_fold[0][cut_idx]['w2']:.2f}" for cut_idx in range(len(cut_s_over_root_bs_fold[0]))
# #     ]
# #     IN_cut_labels = [
# #         f"s/√b={cut_s_over_root_bs_fold[1][cut_idx]:.2f}, s={sig_weights_fold[1][cut_idx]['value']:.2f}±{sig_weights_fold[1][cut_idx]['w2']:.2f}, b={bkg_weights_fold[1][cut_idx]['value']:.2f}±{bkg_weights_fold[1][cut_idx]['w2']:.2f}" for cut_idx in range(len(cut_s_over_root_bs_fold[1]))
# #     ]
# #     line_labels = [BDT_cut_labels, IN_cut_labels]
# #     lines = [cut_boundaries_fold[0], cut_boundaries_fold[1]]
# #     line_colors=[cmap_petroff10, cmap_petroff10]

# #     plot_s_over_root_b(
# #         sigs, bkgs, labels, f"BDT_IN_sOverRootb_withCuts_testData_fold{fold_idx}", plot_dirpath, weights=weights_plot,
# #         lines=lines, lines_labels=line_labels, line_colors=line_colors
# #     )    

# for pred_type, dataset in [
#         ('train_', bdt_train_dict[f"fold_{fold_idx}"]), 
#         ('val_', bdt_val_dict[f"fold_{fold_idx}"]),
#         ('', bdt_test_dict[f"fold_{fold_idx}"])
#     ]:

In [ ]:
xgb.plot_importance(booster)

In [ ]:
import graphviz
xgb.to_graphviz(booster)

In [ ]:
bdtfile = rt.TFile.Open("TMVA.root")
bdttree = bdtfile.Get("TestTree")
bdtscore = tree2array(bdttree,
                        branches = ['classID','BDT'])
bdtframe = pd.DataFrame.from_records(bdtscore)
bdtarray = bdtframe.values

bdt_pred = bdtarray[:,1]
bdt_target = np.zeros(shape=len(bdt_pred))
bdt_target[bdtarray[:,0]==0] = 1 # Signal
bdt_target[bdtarray[:,0]==1] = -1 # Background

fpr_bdt, tpr_bdt, thresholds_bdt = roc_curve(bdt_target, bdt_pred)
area_bdt = auc(fpr_bdt, tpr_bdt)

with h5py.File("BDT_ROC.h5","w") as out:
    out['FPR'] = fpr_bdt
    out['TPR'] = tpr_bdt
    out['Thresholds'] = thresholds_bdt
    print("Saved ROC.")

TPR_thresholds = [0.96, 0.94, 0.935, 0.9, 0.7, 0.5, 0.3]
print("BDT performance")
print("Threshold \tSignal Efficiency \tBackground contamination")
for TPR_threshold in TPR_thresholds:
    thres_bdt_idx = np.argmax(tpr_bdt>TPR_threshold)
    print("{0:.4}  \t   {1:.4}   \t\t  {2:.4}".format(thresholds_bdt[thres_bdt_idx], tpr_bdt[thres_bdt_idx],  fpr_bdt[thres_bdt_idx]))


# TPR_threshold = 0.96
# thres_bdt_idx = np.argmax(tpr_bdt>TPR_threshold)
# print("Signal efficiency = {} @ {} ttH background contamination".format(tpr_bdt[thres_bdt_idx], fpr_bdt[thres_bdt_idx]))
# print("NN score threshold = {}".format(thresholds_bdt[thres_bdt_idx]))

plt.figure(figsize=(9,7))
plt.plot(fpr_bdt,tpr_bdt,label="AUC = {}".format(area_bdt))
plt.legend(loc='best')
plt.xlabel('Background contamination')
plt.ylabel('Signal efficiency')
plt.axhline(tpr_bdt[thres_bdt_idx],ls='--',color='tab:gray')
plt.axvline(fpr_bdt[thres_bdt_idx],ls='--',color='tab:gray')

plt.figure(figsize=(9,7))
plt.hist(bdt_pred[bdt_target==-1], bins=60, label='ttH background',alpha=0.5, normed=True)
plt.hist(bdt_pred[bdt_target==1], bins=60, label='HH signal', alpha=0.5, normed=True)
#plt.axvline(thresholds_bdt[thres_bdt_idx], ls='--',color='tab:gray')
plt.legend(loc='best')